## 1. Importing libraries

In [5]:
import numpy as np
import pandas as pd
from surprise import Dataset, evaluate, accuracy, Reader, dump, get_dataset_dir
import io
from collections import defaultdict
from surprise.model_selection import train_test_split

In [6]:
pd.set_option('display.max_rows',300)
pd.set_option('display.max_columns',300)

## 2. Loading datasets

In [7]:
preds, model = dump.load('KNNfinal_model')

In [8]:
ratings = pd.read_csv('item_item_final.csv')

In [9]:
df = pd.read_csv('final_data.csv')
gb_movies = df[['movieId','title']].copy()

In [10]:
# Combining titles and ratings for recommender. First, getting the titles by movieId. 
ids = list(gb_movies['movieId'])
titles = list(gb_movies['title'])
zip_list = list(zip(ids,titles))
zip_list

[(56, 'Kids of the Round Table (1995)'),
 (66, 'Lawnmower Man 2: Beyond Cyberspace (1996)'),
 (181, 'Mighty Morphin Power Rangers: The Movie (1995)'),
 (193, 'Showgirls (1995)'),
 (285, 'Beyond Bedlam (1993)'),
 (286, 'Nemesis 2: Nebula (1995)'),
 (393, 'Street Fighter (1994)'),
 (397, 'Fear, The (1995)'),
 (470, 'House Party 3 (1994)'),
 (546, 'Super Mario Bros. (1993)'),
 (591, 'Tough and Deadly (1995)'),
 (634, 'Theodore Rex (1995)'),
 (658, "Billy's Holiday (1995)"),
 (737, 'Barb Wire (1996)'),
 (747, 'Stupids, The (1996)'),
 (810, 'Kazaam (1996)'),
 (815, 'Power 98 (1996)'),
 (820, 'Death in the Garden (Mort en ce jardin, La) (1956)'),
 (821, 'Crude Oasis, The (1995)'),
 (826, 'Women Robbers (Diebinnen) (1995)'),
 (834, 'Phat Beach (1996)'),
 (855, 'Every Other Weekend (Un week-end sur deux) (1990)'),
 (870, "Gone Fishin' (1997)"),
 (873, 'Shadow of Angels (Schatten der Engel) (1976)'),
 (878, 'Bye-Bye (1995)'),
 (1105, 'Children of the Corn IV: The Gathering (1996)'),
 (1106, 'Le

In [11]:
# Matching the title list to ratings dataset. 
title_list = []
for i in ratings['movieId']:
    for j in zip_list:
        if i == j[0]:
            title_list.append(j[1])
        else:
            continue

In [12]:
len(title_list)

216399

In [13]:
len(ratings)

216399

In [14]:
ratings['title'] = title_list
ratings.head()

,Unnamed: 0,userId,movieId,rating,timestamp,title
0,66,4,66,0.5,1145937915,Lawnmower Man 2: Beyond Cyberspace (1996)
1,90,4,181,0.5,1127946789,Mighty Morphin Power Rangers: The Movie (1995)
2,93,4,193,0.5,1113766123,Showgirls (1995)
3,159,4,546,0.5,1115782390,Super Mario Bros. (1993)
4,296,4,1760,0.5,1114050751,Spice World (1997)


In [15]:
# Creating final recommendation dataset. 
final = ratings[['userId', 'title', 'rating']]
final.head(125)

,userId,title,rating
0,4,Lawnmower Man 2: Beyond Cyberspace (1996),0.5
1,4,Mighty Morphin Power Rangers: The Movie (1995),0.5
2,4,Showgirls (1995),0.5
3,4,Super Mario Bros. (1993),0.5
4,4,Spice World (1997),0.5
5,4,Air Bud: Golden Receiver (1998),0.5
6,4,Police Academy 4: Citizens on Patrol (1987),0.5
7,4,Howard the Duck (1986),0.5
8,4,Iron Eagle II (1988),2.5
9,4,Stop! Or My Mom Will Shoot (1992),0.5


In [22]:
final.to_csv('KNN_final.csv')

## 3. Creating recommender from imported model

In [16]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(final,reader)
trainset = data.build_full_trainset()

In [17]:
model.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
# Testing a function to pull recommendations from raw_id. 
mm_raw_id = 'House of the Dead, The (2003)'
mm_inner_id = model.trainset.to_inner_iid(mm_raw_id)
mm_neighbors = model.get_neighbors(mm_inner_id, k=5)
# Convert inner ids of the neighbors into names.
mm_raw = [model.trainset.to_raw_iid(inner_id)
                       for inner_id in mm_neighbors]

In [19]:
mm_neighbors

[148, 44, 155, 27, 46]

In [20]:
mm_inner_id

108

In [21]:
mm_raw

['Texas Chainsaw Massacre: The Next Generation (a.k.a. The Return of the Texas Chainsaw Massacre) (1994)',
 'Jason X (2002)',
 'Jason Goes to Hell: The Final Friday (1993)',
 'Kazaam (1996)',
 'Son of the Mask (2005)']

## Recommending

In [249]:
hd_raw_id = 'Friday the 13th Part VII: The New Blood (1988)'
hd_inner_id = KNN_final_model2.trainset.to_inner_iid(hd_raw_id)
hd_neighbors = KNN_final_model2.get_neighbors(hd_inner_id, k=5)
# Convert inner ids of the neighbors into names.
hd_raw = [KNN_final_model2.trainset.to_raw_iid(hd_inner_id)
                       for hd_inner_id in hd_neighbors]

In [250]:
hd_neighbors

[31, 82, 106, 28, 77]

In [256]:
import pprint

In [260]:
pp = pprint.PrettyPrinter(width=40)
print('Recommendations for Friday the 13th Part VII: The New Blood (1988):')
pp.pprint(hd_raw)

Recommendations for Friday the 13th Part VII: The New Blood (1988):
['Friday the 13th Part VIII: Jason '
 'Takes Manhattan (1989)',
 'Leatherface: Texas Chainsaw Massacre '
 'III (1990)',
 'Amityville: A New Generation (1993)',
 'Jaws 3-D (1983)',
 'Texas Chainsaw Massacre: The Next '
 'Generation (a.k.a. The Return of the '
 'Texas Chainsaw Massacre) (1994)']


In [3]:
preds, model = dump.load('KNNfinal_model')

In [4]:
t_raw_id = 'The Hungover Games (2014)'
t_inner_id = model.trainset.to_inner_iid(t_raw_id)
t_neighbors = model.get_neighbors(t_inner_id, k=5)
# Convert inner ids of the neighbors into names.
t_raw = [model.trainset.to_raw_iid(t_inner_id)
                       for t_inner_id in t_neighbors]

In [5]:
t_raw

['Home Alone 3 (1997)',
 'Disaster Movie (2008)',
 'Norbit (2007)',
 'Epic Movie (2007)',
 'Catwoman (2004)']

In [231]:
plot_df = pd.read_csv('with_plots.csv')
plot_df.drop('Unnamed: 0',inplace=True,axis=1)

In [235]:
plot_df.head(250)

,movieId,title,genres,imdbId,tmdbId,"('count', 0.5)","('count', 1.0)","('count', 1.5)","('count', 2.0)","('count', 2.5)","('count', 3.0)","('count', 3.5)","('count', 4.0)","('count', 4.5)","('count', 5.0)",total_count,percent_0.5to1,percent_4.5to5,percent_polarity,total_tails,Cluster,Plot
0,56,Kids of the Round Table (1995),Adventure|Children|Comedy|Fantasy,tt0113541,124057.0,4.0,33.0,2.0,15.0,2.0,11.0,2.0,12.0,0.0,2.0,83.0,0.445783,0.024096,0.421687,0.469880,1,Eleven-year-old Alex and his fellow homemade h...
1,181,Mighty Morphin Power Rangers: The Movie (1995),Action|Children,tt0113820,9070.0,310.0,970.0,99.0,534.0,130.0,602.0,81.0,159.0,24.0,92.0,3001.0,0.426524,0.038654,0.387871,0.465178,1,"Six teenagers Tommy, Kimberly, Adam, Billy, Ro..."
2,286,Nemesis 2: Nebula (1995),Action|Sci-Fi|Thriller,tt0113948,63105.0,8.0,38.0,3.0,30.0,2.0,18.0,2.0,8.0,0.0,5.0,114.0,0.403509,0.043860,0.359649,0.447368,1,It has been 73 years since Alex failed and the...
3,591,Tough and Deadly (1995),Action|Drama|Thriller,tt0114706,80350.0,0.0,14.0,1.0,9.0,0.0,8.0,0.0,3.0,0.0,1.0,36.0,0.388889,0.027778,0.361111,0.416667,1,Elmo Freech is a private investigator and was ...
4,810,Kazaam (1996),Children|Comedy|Fantasy,tt0116756,11511.0,292.0,732.0,97.0,344.0,92.0,379.0,45.0,103.0,3.0,71.0,2158.0,0.474513,0.034291,0.440222,0.508804,1,"Being a lone young boy in the 'hood"" is danger..."
5,1322,Amityville 1992: It's About Time (1992),Horror,tt0103678,41671.0,28.0,138.0,7.0,52.0,10.0,46.0,6.0,12.0,1.0,9.0,309.0,0.537217,0.032362,0.504854,0.569579,1,"When an old clock arrives at home as a gift, s..."
6,1323,Amityville 3-D (1983),Horror,tt0085159,27214.0,48.0,203.0,24.0,96.0,8.0,50.0,4.0,17.0,1.0,8.0,459.0,0.546841,0.019608,0.527233,0.566449,1,A man who is separated from his wife moves int...
7,1324,Amityville: Dollhouse (1996),Horror,tt0115535,52263.0,18.0,117.0,2.0,45.0,10.0,35.0,3.0,13.0,1.0,9.0,253.0,0.533597,0.039526,0.494071,0.573123,1,A dollhouse that is a replica of the infamous ...
8,1325,Amityville: A New Generation (1993),Horror,tt0106262,33519.0,32.0,129.0,8.0,56.0,4.0,46.0,4.0,12.0,1.0,5.0,297.0,0.542088,0.020202,0.521886,0.562290,1,Photographer Keyes is given an old mirror from...
9,1326,Amityville II: The Possession (1982),Horror,tt0083550,16235.0,31.0,222.0,26.0,136.0,29.0,130.0,18.0,27.0,4.0,16.0,639.0,0.395931,0.031299,0.364632,0.427230,1,An Italian-American family move into a house b...
